In [2]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

In [3]:
import sys
sys.path.insert(0,'../')

In [6]:
import image_utils.image_thresholder as it
import image_utils.region_masker as rm
import image_utils.poly_fitter as pf
import image_utils.line as line
import pipeline.image_processor as ip

In [ ]:
#np.dstack((it.combined_thresh(rgb_img=image), np.zeros_like(image), np.zeros_like(image)))*255

In [ ]:
#funk = it.combined_thresh
#funk = lambda frame: np.dstack((it.combined_thresh(rgb_img=frame), np.zeros_like(frame), np.zeros_like(frame)*255))

In [ ]:
mtx = np.load('../camera_calibration/saved_data_to_calibrate_images/mtx.npy')
dist = np.load('../camera_calibration/saved_data_to_calibrate_images/dist.npy')
img_proc = ip.ImageProcessor(calibration_matrix=mtx, distortion_coefficients=dist)

In [7]:
left_line = line.Line()
right_line = line.Line()

In [8]:
left_line

In [ ]:
def make_slr_binary_img(rgb_frame):
    i1 = it.combined_thresh(rgb_img=rgb_frame)
    i2 = np.zeros_like(i1)
    i3 = np.zeros_like(i1)
    return np.dstack((i1, i2, i3))*255
def make_slr_reg_mask_binary_img(rgb_frame):
    binary_img = it.combined_thresh(rgb_img=rgb_frame)
    i1 = rm.make_region_of_interest(gray_image=binary_img)
    i2 = np.zeros_like(i1)
    i3 = np.zeros_like(i1)
    return np.dstack((i1, i2, i3))*255
def make_slr_rm_perspec_trans(rgb_frame):
    i1 = img_proc.process_image(rgb_frame)
    i2 = np.zeros_like(i1)
    i3 = np.zeros_like(i1)
    return np.dstack((i1, i2, i3))*255
def make_simple_polyfit_pt(rgb_frame):
    binary_img = img_proc.process_image(rgb_frame)
    i1 = binary_img
    i2 = np.zeros_like(i1)
    i3 = np.zeros_like(i1)
    rgb_ish_image = np.dstack((i1, i2, i3))*255
    
    left_poly, right_poly = pf.simple_polyfit(grayscale_image=binary_img)
    if left_poly is not None and right_poly is not None:
        ploty = np.linspace(0, binary_img.shape[0]-1, binary_img.shape[0] )
        left_fitx = left_poly[0]*ploty**2 + left_poly[1]*ploty + left_poly[2]
        right_fitx = right_poly[0]*ploty**2 + right_poly[1]*ploty + right_poly[2]
        
        
        
        ploty = np.int32(ploty)
        left_fitx = np.int32(left_fitx)
        right_fitx = np.int32(right_fitx)
        for i in range(1, len(ploty)):
            cv2.line(rgb_ish_image, (left_fitx[i-1], ploty[i-1]), (left_fitx[i], ploty[i]), color=(255,255,0), thickness=3)
            cv2.line(rgb_ish_image, (right_fitx[i-1], ploty[i-1]), (right_fitx[i], ploty[i]), color=(255,255,0), thickness=3)
    return rgb_ish_image

In [ ]:
project_video_name = "project_video.mp4"
save_file_name = '../output_videos/perspec_trans_outputs/simple_polyfit/' + project_video_name
src_video_name = "../test_videos/" + project_video_name
clip1 = VideoFileClip(src_video_name)
srl_clip = clip1.fl_image(make_simple_polyfit_pt)
%time srl_clip.write_videofile(save_file_name, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(save_file_name))